In [2]:
# COMP30027 Machine learning Project 1: Gaining Information about Naive Bayes
# Author: Jordan Ung <jordanu@student.unimelb.edu.au> [729938]
# Last Modified: 26.03.19

#### GENERAL GUIDE
# Read In And Inspect The Data
# Check for missing value - (1) delete rows with missing values, (2) Impute the missing values with dataset
# Check for anomalously extreme values

# QUESTIONS TO ASK:
# Should I handle datasets with Missing values and ones without differently?
# In tht case do I need to hardcode it or iterate through the whole dataset once
# to find presence of missing values?

# Is hardcoding bad: ->See "?", is it okay if I add another
# Parameter to the function?

In [3]:
# Preprocess takes the name of a file and returns a list of instances
# within that file, with each instance containing a list of attributes
def preprocess(file_name):
    dataset = []
    with open(file_name, 'r') as file:
        # Add each instance to a list to be used later
        for line in file.readlines():
            dataset.append(line.strip().split(','))
            
    # Group all the instances with the same class together
    dataset = sorted(dataset, key=lambda x: x[-1])
    return dataset

data_file = 'anneal.csv'
missing_value_files = ['breast-cancer.csv', 'hepatitis.csv', 'hypothyroid.csv', 'mushroom.csv', 'primary-tumor.csv']
# preprocess(data_file)

In [39]:
# Train takes a list of instances and returns a 2-tuple, containing
# a list of counts representing the class distribution and a list
# of lists of dictionaries representing the conditional probabilities
def train(instance_list, missing_value):
    data_info = ([], [])
    classes = {}
    
    for instance in instance_list:
        # Check for missing values and delete rows containing them
        if missing_value in instance:
            instance_list.remove(instance)
            
        class_type = instance[-1]
        # Calculate the frequencies of all the class values
        if class_type not in classes:
            classes[class_type] = 1
        else:
            classes[class_type] += 1
            
    # Create a data structure to store all the attribute probabilities w.r.t each class
    for i in range(len(classes)):
        data_info[1].append([])
        for j in range(len(instance_list[0]) - 1):
            data_info[1][i].append({})

    # Calculate probability of each class value compared to total number of instances
    num_of_instances = len(instance_list)
    for type in classes:
        data_info[0].append(classes[type] * 1.0 / num_of_instances)  
        
    current_class = instance_list[0][-1]
    class_index = 0
    # Calculate probability of each value in each attribute for each class
    for data in instance_list:
        attribute_num = 0
        
        # Input each instance's attribute into the appropriate dictionary
        for attribute in data[:-1]:
            if attribute in data_info[1][class_index][attribute_num]:
                data_info[1][class_index][attribute_num][attribute] += 1
            else:
                data_info[1][class_index][attribute_num][attribute] = 1
            attribute_num += 1
            
    return data_info # Return the 2-tuple
train(preprocess(missing_value_files[0]), "?")
# train(preprocess(data_file))

([0.7075812274368231, 0.2924187725631769],
 [[{'20-29': 1,
    '30-39': 36,
    '40-49': 89,
    '50-59': 91,
    '60-69': 55,
    '70-79': 5},
   {'ge40': 123, 'lt40': 5, 'premeno': 149},
   {'0-4': 8,
    '10-14': 28,
    '15-19': 29,
    '20-24': 48,
    '25-29': 51,
    '30-34': 57,
    '35-39': 19,
    '40-44': 22,
    '45-49': 3,
    '5-9': 4,
    '50-54': 8},
   {'0-2': 209,
    '12-14': 3,
    '15-17': 6,
    '24-26': 1,
    '3-5': 34,
    '6-8': 17,
    '9-11': 7},
   {'no': 221, 'yes': 56},
   {'1': 66, '2': 129, '3': 82},
   {'left': 145, 'right': 132},
   {'central': 21,
    'left_low': 106,
    'left_up': 94,
    'right_low': 23,
    'right_up': 33},
   {'no': 215, 'yes': 62}],
  [{}, {}, {}, {}, {}, {}, {}, {}, {}]])

In [69]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instance):
    return # Return a class

In [5]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate():
    return

In [6]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain():
    return

[[], [], [], [], [], [], [], [], [], []]


In [76]:
d = [{},{}]
s = d[1]
s['name'] = 'Jordan'
print(d)


[{}, {'name': 'Jordan'}]
